# Final Project - Training with optimization.

In [1]:
import tensorflow as tf
import os
import numpy as np
import random
import pandas as pd

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

2023-06-11 14:46:42.684839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 14:46:42.839894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-11 14:46:42.839938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-11 14:46:42.882882: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-11 14:46:44.644646: W tensorflow/stream_executor/pla

In [2]:
PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.016,
    'frame_step_in_s': 0.012,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 8000,
    'num_coefficients': 30
}
TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 0,
    'epochs': 20
}

final_sparsity = 0.7 #[0.7; 0.95]
alpha = 0.4 #[0.25; 0.5; 0.75]

## Preprocessing

In [3]:
from loader import *
data_loader = VISCDataLoader(seconds = 2, channels = 1, feature_type='MFCCS', **PREPROCESSING_ARGS)


In [4]:
batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = data_loader.get_split('train')
train_ds = train_ds.cache()
train_ds = train_ds.batch(batch_size)
print(train_ds)
val_ds = data_loader.get_split('val').batch(batch_size)
test_ds = data_loader.get_split('test').batch(batch_size)

2023-06-11 14:46:47.902390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-11 14:46:47.902424: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-11 14:46:47.902445: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-05dea53b-e624-4a45-a48c-4f8a58ba823b): /proc/driver/nvidia/version does not exist
2023-06-11 14:46:47.902728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 14:46:48.610963: W tensorflow_io/core/kernels/audio_video_mp3_ker

In [5]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 32, 32, 1)
Data Shape: (32, 32, 1)
Labels: tf.Tensor([4 7 1 3 0 0 7 7 5 7 6 0 0 7 0 2 3 0 6 2], shape=(20,), dtype=int32)
2023-06-11 14:46:54.191309: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## DS-CNN

Conv2D(filters=(128*alpha), kernel_size=[3, 3], stride=[2, 2],use_bias=False, padding=’valid’)
BatchNormalization()
ReLU()
DepthwiseConv2D(kernel_size=[3, 3], stride=[1, 1],
use_bias=False, padding=’same’)
Conv2D(filters=(128*alpha), kernel_size=[1, 1], stride=[1, 1],use_bias=False)
BatchNormalization()
ReLU()
DepthwiseConv2D(kernel_size=[3, 3], stride=[1, 1],
use_bias=False, padding=’same’)
Conv2D(filters=(128*alpha), kernel_size=[1, 1], stride=[1, 1],use_bias=False)
BatchNormalization()
ReLU()
GlobalAveragePooling2D()
Dense(units=len(LABELS)
Softmax()

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=(128*alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=(128*alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=(128*alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=8),
    tf.keras.layers.Softmax()
])

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 15, 15, 51)        459       
                                                                 
 batch_normalization_3 (Batc  (None, 15, 15, 51)       204       
 hNormalization)                                                 
                                                                 
 re_lu_3 (ReLU)              (None, 15, 15, 51)        0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 15, 15, 51)       459       
 Conv2D)                                                         
                                                                 
 conv2d_4 (Conv2D)           (None, 15, 15, 51)        2601      
                                                                 
 batch_normalization_4 (Batc  (None, 15, 15, 51)      

## Setup Magnitude-based Weights Pruning

In [9]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs*0.8)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

## Training

In [10]:
#Run this only if you're not pruning
#model_for_pruning = model

In [11]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Epoch 1/20
209/209 [==============================] - 113s 521ms/step - loss: 0.9053 - sparse_categorical_accuracy: 0.7001 - val_loss: 2.1296 - val_sparse_categorical_accuracy: 0.4759
Epoch 2/20
209/209 [==============================] - 24s 116ms/step - loss: 0.3476 - sparse_categorical_accuracy: 0.8878 - val_loss: 0.3406 - val_sparse_categorical_accuracy: 0.8666
Epoch 3/20
209/209 [==============================] - 24s 116ms/step - loss: 0.2321 - sparse_categorical_accuracy: 0.9284 - val_loss: 0.2012 - val_sparse_categorical_accuracy: 0.9196
Epoch 4/20
209/209 [==============================] - 24s 116ms/step - loss: 0.1744 - sparse_categorical_accuracy: 0.9431 - val_loss: 0.1436 - val_sparse_categorical_accuracy: 0.9518
Epoch 5/20
209/209 [==============================] - 24s 116ms/step - loss: 0.1183 - sparse_categorical_accuracy: 0.9618 - val_loss: 0.1233 - val_sparse_categorical_accuracy: 0.9598
Epoch 6/20
209/209 [==============================] - 24s 116ms/step - loss: 0.1137 

## Testing

In [12]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

60/60 [==============================] - 28s 455ms/step - loss: 0.0547 - sparse_categorical_accuracy: 0.9858


## Recap

In [13]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.0238
Training Accuracy: 99.59%

Validation Loss: 0.0275
Validation Accuracy: 99.36%

Test Loss: 0.0547
Test Accuracy: 98.58%


In [14]:
import pandas as pd
from time import time

timestamp = int(time())

output_dict = {
    'timestamp': timestamp,
    'feature_type': data_loader.get_feature_type(),
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'test_accuracy': test_accuracy,
}

df = pd.DataFrame([output_dict])

output_path='./mfccs_spectrogram_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

## Pruning

In [15]:
from time import time

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
MODEL_NAME = 'model_1600hz2'

saved_model_dir = f'./{MODEL_NAME}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model_for_export.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./model_1600hz2/assets
INFO:tensorflow:Assets written to: ./model_1600hz2/assets


# Compression

In [16]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./{MODEL_NAME}')

tflite_model = converter.convert()

2023-06-11 14:57:26.933110: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-11 14:57:26.933158: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-11 14:57:26.933868: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./model_1600hz2
2023-06-11 14:57:26.937645: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-11 14:57:26.937676: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./model_1600hz2
2023-06-11 14:57:26.945378: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-06-11 14:57:26.947110: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-06-11 14:57:26.987413: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./model_1600hz2
2023-06-11

In [17]:
tflite_models_dir = './models/tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [18]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

'./models/tflite_models/model_1600hz2.tflite'

In [19]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

## Zipping

In [20]:
import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name, f'{MODEL_NAME}.tflite')

In [21]:
tflite_size = os.path.getsize(tflite_model_name) / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size (pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')

Original tflite size (pruned model): 31.961 KB
Zipped tflite size (pruned model): 15.159 KB


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05dea53b-e624-4a45-a48c-4f8a58ba823b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>